This example solves the same AR(1) problem, but the purpose here is to demonstration the versatitily of `linkalman`. It has the following three changes:

1. three different methods to illustrate all three optimizing techniques
2. use customized ft with BaseOpt instead of BaseConstantModel
3. use nlopt as an alternative solver to illustrate flexibility of linkalman solver config

In [25]:
"""
AR(1) model
"""
import numpy as np
import pandas as pd
import linkalman
import scipy
from linkalman.models import BaseOpt as BM
from linkalman.core.utils import build_tensor
import nlopt
%matplotlib inline 


# Initialization
Instead of relying on `BaseConstantModel`, here I directly define the function `my_ft`. An `ft` should have two required positional arguments: `theta` and `T`. It may also contain keyword arguments. In defining a constant model, I also leverage the `linkalman.core.utils.Constant_M` module. If you have a long time series with mostly constant system dynamic matrices, you can use `Constant_M` to save storage spaces. 

In [26]:
def my_ft(theta, T, x_0=0):
    """
    AR(1) model with noise. In general, MLE is biased, so the focus should be 
    more on prediction fit, less on parameter estimation. The 
    formula here for Ar(1) is:
    y_t = c + Fy_{t-1} + epsilon_{t-1}
    """
    # Define theta
    phi_1 = 1 / (np.exp(theta[0])+1)
    sigma = np.exp(theta[1]) 
    sigma_R = np.exp(theta[3]) 
    # Generate F
    F = np.array([[phi_1]])
    # Generate Q
    Q = np.array([[sigma]]) 
    # Generate R
    R = np.array([[sigma_R]])
    # Generate H
    H = np.array([[1]])
    # Generate B
    B = np.array([[theta[2]]])
    # Generate D
    D = np.array([[0]])
    
    # Build Mt
    Ft = build_tensor(F, T)
    Bt = build_tensor(B, T)
    Qt = build_tensor(Q, T)
    Ht = build_tensor(H, T)
    Dt = build_tensor(D, T)
    Rt = build_tensor(R, T)
    xi_1_0 = theta[2] * x_0 / (1 - phi_1)  # calculate stationary mean, x_0 is already np.ndarray
    P_1_0 = np.array([[sigma /(1 - phi_1 * phi_1)]])  # calculate stationary cov
    
    Mt = {'Ft': Ft,
          'Bt': Bt,
          'Qt': Qt,
          'Ht': Ht,
          'Dt': Dt,
          'Rt': Rt,
          'xi_1_0': xi_1_0,
          'P_1_0': P_1_0}
    return Mt


# Solver

In [27]:
def my_solver(param, obj_func, **kwargs):
    """
    More complex solver function than the simple AR(1) case.
    The purpose is to provide an example of flexiblity of
    building solvers. Note I also suppress grad in nlopt_obj, 
    as linkalman uses only non-gradient optimizers
    """
    def nlopt_obj(x, grad, **kwargs):
        fval_opt = obj_func(x)
        if kwargs.get('verbose', False):
            print('fval: {}'.format(fval_opt))
        return fval_opt

    opt = nlopt.opt(nlopt.LN_BOBYQA, param.shape[0])
    obj = lambda x, grad: nlopt_obj(x, grad, **kwargs)
    opt.set_max_objective(obj)
    opt.set_xtol_rel(kwargs.get('xtol_rel', opt.get_xtol_rel()))
    opt.set_ftol_rel(kwargs.get('ftol_rel', opt.get_ftol_rel()))
    theta_opt = opt.optimize(param)
    fval_opt = opt.last_optimum_value()
    if kwargs.get('verbose_opt', False):
        print('fval: {}'.format(fval_opt))
        
    return theta_opt, fval_opt

In [28]:
# Initialize the model
x = 1
model = BM()
model.set_f(my_ft, x_0=x * np.ones([1, 1]))
model.set_solver(my_solver, xtol_rel=1e-4, verbose=True) 


# Generate Synthetic Data

In [29]:
# Some initial parameters
theta = np.array([0, -0.1, 0.1, 1])
T = 2000
train_split_ratio = 0.7
forecast_cutoff_ratio = 0.8  

# Split train data
train_split_t = np.floor(T * train_split_ratio).astype(int)

# Generate missing data for forcasting
forecast_t = np.floor(T * forecast_cutoff_ratio).astype(int)

# If we want AR(1) with non-zero stationary mean, we should proivde a constant 
x_col = ['const']
Xt = pd.DataFrame({x_col[0]: x * np.ones(T)})  # use x to ensure constant model

# Build simulated data
df, y_col, xi_col = model.simulated_data(input_theta=theta, Xt=Xt)

# Store fully visible y for comparison later
df['y_0_vis'] = df.y_0.copy()  

# Splits models into three groups
is_train = df.index < train_split_t
is_test = (~is_train) & (df.index < forecast_t)
is_forecast = ~(is_train | is_test)

# Create a training and test data
df_train = df.loc[is_train].copy()

# Build two kinds of test data (full data vs. test data only)
df_test = df.copy()  

# Create an offset
df_test.loc[is_forecast, ['y_0']] = np.nan

# LLY
First, I use numerical methods. It is the preferred methods over EM algorithm, because EM need score function to be effective, which is rather limiting. `linkalman` makes a compromise to gain flexibility in handling missing measurements and customized `ft`.

In [30]:
# Fit data using LLY:
theta_init = np.random.rand(len(theta))
model.fit(df_train, theta_init, y_col=y_col, x_col=x_col, 
              method='LLY')
theta_LLY = model.theta_opt

fval: -3223.780353605801
fval: -3234.561809853953
fval: -3310.260280868972
fval: -3328.1984100977324
fval: -3456.636793173768
fval: -3242.1573106794576
fval: -3234.992905247796
fval: -3222.239100224533
fval: -3256.2655883564653
fval: -3216.4788739126393
fval: -3196.55788615726
fval: -3362.222650684933
fval: -3201.0924426590236
fval: -3202.440196796843
fval: -3201.603496585028
fval: -3196.48703399456
fval: -3199.368877681782
fval: -3193.9130548571093
fval: -3194.4071415738267
fval: -3194.2146663545977
fval: -3195.213923446289
fval: -3193.54681352407
fval: -3193.299890352676
fval: -3193.209470000652
fval: -3193.199772660622
fval: -3193.171337199446
fval: -3193.139096795467
fval: -3193.1091800629792
fval: -3193.1466962155464
fval: -3193.1512267731837
fval: -3193.0896114881984
fval: -3193.0893062262576
fval: -3193.0904573076787
fval: -3193.0812225480763
fval: -3193.082681416268
fval: -3193.0942540000315
fval: -3193.0847272653205
fval: -3193.0818193474447
fval: -3193.0915999676445
fval: -31

# Use EM to Cold Start Optimization
One may combine EM and LLY method. EM methods, with score functions have very fast convergence at the beginning. Interested readers may design their own solvers to compute the saddle point directly. 

In [31]:
# Fit data using both methods:
theta_init = np.random.rand(len(theta))
model.set_solver(my_solver, xtol_rel=1e-3, ftol_rel=1e-3, verbose_opt=True) 
model.fit(df_train, theta_init, y_col=y_col, x_col=x_col, method='EM', num_EM_iter=20)
model.set_solver(my_solver, xtol_rel=1e-4, verbose=True) 
model.fit(df_train, model.theta_opt, y_col=y_col, x_col=x_col, method='LLY')
theta_mix = model.theta_opt

fval: -3973.78189067131
fval: -4081.248158034627
fval: -4191.753625029391
fval: -4208.026969528717
fval: -4223.2025343844825
fval: -4235.305600219138
fval: -4245.718627942523
fval: -4253.107876202973
fval: -4258.003504280382


KeyboardInterrupt: 

# EM
In order to use EM algorithm numerically, one has to be careful about tolerance. If the tolerence parameters are too small, it's slow to finish one iteration. On the other hand, due to the iterative nature of EM algorithm, if one set the tolerance too large, it will fail to converge later on. Therefore, EM is better for cold starting a optimizer. 

In [ ]:
# Fit data using EM:
theta_init = np.random.rand(len(theta))
model.set_solver(my_solver, xtol_rel=1e-5, ftol_rel=1e-5, verbose_opt=True) 
model.fit(df_train, theta_init, y_col=y_col, x_col=x_col, EM_threshold=0.005, method='EM')
theta_EM = model.theta_opt

In [ ]:
# Make predictions from LLY:
df_LLY = model.predict(df_test, theta=theta_LLY)

# Make predictions from mixed models:
df_mix = model.predict(df_test, theta=theta_mix)

# Make predictions from EM:
df_EM = model.predict(df_test, theta=theta_EM)

# Make predictions using true theta:
df_true = model.predict(df_test, theta=theta)

# Compare Performance

In [ ]:
# Calculate Statistics
RMSE = {}
RMSE['true'] = np.sqrt((df_true.y_0_filtered - df_true.y_0_vis).var())
RMSE['LLY'] = np.sqrt((df_LLY.y_0_filtered - df_LLY.y_0_vis).var())
RMSE['EM'] = np.sqrt((df_EM.y_0_filtered - df_EM.y_0_vis).var())
RMSE['mix'] = np.sqrt((df_mix.y_0_filtered - df_mix.y_0_vis).var())

M_error = {}
M_error['true'] = (df_true.y_0_filtered - df_true.y_0_vis).mean()
M_error['LLY'] = (df_LLY.y_0_filtered - df_LLY.y_0_vis).mean()
M_error['EM'] = (df_EM.y_0_filtered - df_EM.y_0_vis).mean()
M_error['mix'] = (df_mix.y_0_filtered - df_mix.y_0_vis).mean()
print(RMSE)
print(M_error)